<a href="https://colab.research.google.com/github/rashida048/Apache-Beam/blob/main/Bank_Deafulters_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.9/648.9 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 111.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 27.4 MB/s eta 0:00:00


In [3]:
import apache_beam as beam

In [5]:
p = beam.Pipeline()

def calculate_points(element):
  #print(element)
  li= element.split(',')
  customer_id, first_name, last_name, relationship_id, card_type, max_limit, spent, cash_withdrawn, payment_cleared, payment_date = li[0], li[1], li[2], li[3], li[4], li[5], li[6], li[7], li[8], li[9]

  spent = int(spent)
  payment_cleared = int(payment_cleared)
  max_limit = int(max_limit)

  key_name = customer_id + ', ' + first_name + ' ' + last_name
  defaulter_points = 0

  if payment_cleared < (spent * 0.7):
    defaulter_points += 1

  if (spent == max_limit) and (payment_cleared < spent):
    defaulter_points += 1

  if (spent == max_limit) and (payment_cleared < (spent*0.7)):
    defaulter_points += 1

  return key_name, defaulter_points


def format_results(sum_pair):
  key_name, points = sum_pair

  return str(key_name) + ', ' + str(points) + ' fraud_points'

card_defaulter = (p
                  | 'Read credit card data' >> beam.io.ReadFromText('/content/cards.txt', skip_header_lines=1)
                  | 'Calculate defaulter points' >> beam.Map(calculate_points)
                  |'Combine points for defaulter' >> beam.CombinePerKey(sum)
                  | 'Filter card defaulter' >> beam.Filter(lambda element: element[1] > 0)
                  | 'Format output' >> beam.Map(format_results)
                  | 'Write credit card data' >> beam.io.WriteToText('card_skippers')
                  )
p.run()

In [4]:
!head -n 7 card_skippers-00000-of-00001

CT28383, Miyako Burns, 3 fraud_points
CT74474, Nanaho Brennan, 3 fraud_points
CT66322, Chris Bruce, 1 fraud_points
CT65528, Bonnie Barlow, 2 fraud_points
CT84463, Isaac Bowman, 4 fraud_points
CT12838, Isidore Albert, 5 fraud_points
CT96185, Danielle Bradshaw, 3 fraud_points


In [6]:
with open('card_skippers'+ '-00000-of-00001') as file:
  points_list = []
  for line in file:
    customer_id, key_name, points = line.split(',')
    points = points.rstrip().lstrip()
    point, fraud = points.split(' ')
    tuple_temp = (customer_id+','+key_name, point)

    points_list.append(tuple_temp)
  sorted_point_list = sorted(points_list, key = lambda x: x[1], reverse = True)

  print("Card Defaulter list")

  for i in sorted_point_list:
    key_name, points = i
    customer_id, name = key_name.split(',')
    print(customer_id + ',' + name + ', ' + str(points)+ ' fraud_points')

Card Defaulter list
CT24995, Hitomi Ballard, 8 fraud_points
CT70382, Dolly Acevedo, 8 fraud_points
CT65343, Rafael Bradley, 8 fraud_points
CT18540, Erika Blevins, 8 fraud_points
CT94157, Tanya Browning, 7 fraud_points
CT94260, Akane Calderon, 7 fraud_points
CT21675, Paloma Bryant, 7 fraud_points
CT96252, Nate Burch, 7 fraud_points
CT94340, Danielle Armstrong, 7 fraud_points
CT39897, Chantal Barlow, 7 fraud_points
CT75042, Tomas Avery, 7 fraud_points
CT12879, Valerie Allison, 7 fraud_points
CT18711, Kazuko Becker, 7 fraud_points
CT51720, Nate Beard, 7 fraud_points
CT46579, Michael Bush, 7 fraud_points
CT13093, Frances Buchanan, 7 fraud_points
CT53460, Arthur Carson, 7 fraud_points
CT79250, Maria Bowman, 7 fraud_points
CT53191, Danielle Buck, 7 fraud_points
CT28438, Sandy Adkins, 7 fraud_points
CT94994, Wilfred Baker, 7 fraud_points
CT36498, Huto Bell, 7 fraud_points
CT51282, Richard Adkins, 7 fraud_points
CT79552, Aki Burch, 7 fraud_points
CT98854, Harvey Ball, 7 fraud_points
CT25287, T

In [7]:
import datetime
def calculate_late_payments(element):
  due_date = datetime.datetime.strptime(str(element[6].rstrip().lstrip()), '%d-%m-%Y')
  payment_date = datetime.datetime.strptime(str(element[8].rstrip().lstrip()), '%d-%m-%Y')

  if payment_date <= due_date:
    element.append('0')
  else:
    element.append('1')
  return element

def format_output(sum_pair):
  key_name, miss_months = sum_pair
  return str(key_name) + ', ' + str(miss_months) + ' missed'

In [17]:
def calculate_month(input_list):
  payment_date = datetime.datetime.strptime(str(input_list[8].rstrip().lstrip()), '%d-%m-%Y')
  input_list.append(str(payment_date.month))
  return input_list

def calculate_personal_loan_defaulter(input):
  max_allowed_missed_months = 4
  max_allowed_consecutive_missing = 2

  name, months_list = input
  #print(months_list)
  #months_list.sort()
  sorted_months = sorted(months_list)
  #print(sorted_months)
  total_payments = len(sorted_months)

  missed_payments = 12 - total_payments

  if missed_payments > max_allowed_missed_months:
    return name, missed_payments

  consecutive_missed_months = 0

  temp = sorted_months[0] - 1
  if temp > consecutive_missed_months:
    consecutive_missed_months = temp

  temp = 12 - sorted_months[total_payments -1]
  if temp > consecutive_missed_months:
    consecutive_missed_months = temp

  for i in range(1, len(sorted_months)):
    temp = sorted_months[i] - sorted_months[i-1]-1
    if temp > consecutive_missed_months:
      consecutive_missed_months = temp

  if consecutive_missed_months > max_allowed_consecutive_missing:
    return name, consecutive_missed_months

  return name, 0

In [18]:
p = beam.Pipeline()

card_defaulter = (p
                  | 'Read credit card data' >> beam.io.ReadFromText('/content/cards.txt', skip_header_lines=1)
                  | 'Calculate defaulter points' >> beam.Map(calculate_points)
                  |'Combine points for defaulter' >> beam.CombinePerKey(sum)
                  | 'Filter card defaulter' >> beam.Filter(lambda element: element[1] > 0)
                  | 'Format output' >> beam.Map(format_results)
                  | 'Write credit card data' >> beam.io.WriteToText('card_skippers')
                  )

medical_loan_defaulter = (p
                          | beam.io.ReadFromText('loan.txt', skip_header_lines=1)
                          | 'Split Row' >> beam.Map(lambda row: row.split(','))
                          | 'Filter medical loan' >> beam.Filter(lambda element: (element[5]).rstrip().lstrip() == 'Medical Loan')
                          | 'Calculate late payment' >> beam.Map(calculate_late_payments)
                          | 'Make key value pairs' >> beam.Map(lambda elements: (elements[0] + ', '+ elements[1]+ ' '+elements[2], int(elements[9])))
                          |  'Group medical loan based on month' >> beam.CombinePerKey(sum)
                          | 'Check for medical loan defaulter' >> beam.Filter(lambda element: element[1] >= 3)
                          | 'Format medical loan output' >> beam.Map(format_output)
                          )
personal_loan_defaulter = (p
                           | 'Read' >> beam.io.ReadFromText('loan.txt', skip_header_lines=1)
                           | 'Split' >> beam.Map(lambda row: row.split(','))
                           | 'Filter personal loan' >> beam.Filter(lambda element: (element[5]).rstrip().lstrip() =='Personal Loan')
                           | 'Split and Append new month column' >> beam.Map(calculate_month)
                           | 'Make key value pairs loan' >> beam.Map(lambda elements: (elements[0]+', '+elements[1]+ ' '+elements[2], int(elements[9])))
                           | "Group personal loan based on month" >> beam.GroupByKey()
                           | 'Check for personal loan defaulter' >> beam.Map(calculate_personal_loan_defaulter)
                           | 'Filter only personal loan defaulters' >> beam.Filter(lambda element: element[1] > 0)
                           | 'Format personal loan output' >> beam.Map(format_output)
                           )

final_loan_defaulters = (
    (personal_loan_defaulter, medical_loan_defaulter)
    | 'Combine all defaulters' >> beam.Flatten()
    | 'Write all defaulters to text file' >> beam.io.WriteToText('loan_defaulters')
)
p.run()

In [19]:
!head -n 8 loan_defaulters-00000-of-00001

CT68554, Ronald Chiki, 3 missed
CT56276, Fay Carr, 10 missed
CT30950, Arlene Calderon, 10 missed
CT27126, Nicole Acevedo, 6 missed
CT29233, Wilma Abbott, 5 missed
CT74474, Nanaho Brennan, 5 missed
CT27486, Virginie Ashley, 7 missed
CT37576, Stan Alvarado, 3 missed


In [20]:
def return_tuple(element):
  thisTuple=element.split(',')
  return (thisTuple[0], thisTuple[1:])

In [21]:
p = beam.Pipeline()

card_defaulter = (p
                  | 'Read credit card data' >> beam.io.ReadFromText('/content/cards.txt', skip_header_lines=1)
                  | 'Calculate defaulter points' >> beam.Map(calculate_points)
                  |'Combine points for defaulter' >> beam.CombinePerKey(sum)
                  | 'Filter card defaulter' >> beam.Filter(lambda element: element[1] > 0)
                  | 'Format output' >> beam.Map(format_results)
                  #| 'Write credit card data' >> beam.io.WriteToText('card_skippers')
                  |'tuple' >> beam.Map(return_tuple)
                  )

medical_loan_defaulter = (p
                          | beam.io.ReadFromText('loan.txt', skip_header_lines=1)
                          | 'Split Row' >> beam.Map(lambda row: row.split(','))
                          | 'Filter medical loan' >> beam.Filter(lambda element: (element[5]).rstrip().lstrip() == 'Medical Loan')
                          | 'Calculate late payment' >> beam.Map(calculate_late_payments)
                          | 'Make key value pairs' >> beam.Map(lambda elements: (elements[0] + ', '+ elements[1]+ ' '+elements[2], int(elements[9])))
                          |  'Group medical loan based on month' >> beam.CombinePerKey(sum)
                          | 'Check for medical loan defaulter' >> beam.Filter(lambda element: element[1] >= 3)
                          | 'Format medical loan output' >> beam.Map(format_output)
                          )
personal_loan_defaulter = (p
                           | 'Read' >> beam.io.ReadFromText('loan.txt', skip_header_lines=1)
                           | 'Split' >> beam.Map(lambda row: row.split(','))
                           | 'Filter personal loan' >> beam.Filter(lambda element: (element[5]).rstrip().lstrip() =='Personal Loan')
                           | 'Split and Append new month column' >> beam.Map(calculate_month)
                           | 'Make key value pairs loan' >> beam.Map(lambda elements: (elements[0]+', '+elements[1]+ ' '+elements[2], int(elements[9])))
                           | "Group personal loan based on month" >> beam.GroupByKey()
                           | 'Check for personal loan defaulter' >> beam.Map(calculate_personal_loan_defaulter)
                           | 'Filter only personal loan defaulters' >> beam.Filter(lambda element: element[1] > 0)
                           | 'Format personal loan output' >> beam.Map(format_output)
                           )

final_loan_defaulters = (
    (personal_loan_defaulter, medical_loan_defaulter)
    | 'Combine all defaulters' >> beam.Flatten()
    #| 'Write all defaulters to text file' >> beam.io.WriteToText('loan_defaulters')
    | 'tuple for loan' >> beam.Map(return_tuple)
)

both_defaulers = (
    {'card_defaulters': card_defaulter, 'loan_defaulter': final_loan_defaulters}
    | beam.CoGroupByKey()
    | "Write p3 results" >> beam.io.WriteToText("total")
)

p.run()

In [22]:
!head -n 12 total-00000-of-00001

('CT28383', {'card_defaulters': [[' Miyako Burns', ' 3 fraud_points']], 'loan_defaulter': []})
('CT74474', {'card_defaulters': [[' Nanaho Brennan', ' 3 fraud_points']], 'loan_defaulter': [[' Nanaho Brennan', ' 5 missed']]})
('CT66322', {'card_defaulters': [[' Chris Bruce', ' 1 fraud_points']], 'loan_defaulter': [[' Chris Bruce', ' 8 missed']]})
('CT65528', {'card_defaulters': [[' Bonnie Barlow', ' 2 fraud_points']], 'loan_defaulter': []})
('CT84463', {'card_defaulters': [[' Isaac Bowman', ' 4 fraud_points']], 'loan_defaulter': [[' Isaac Bowman', ' 3 missed']]})
('CT12838', {'card_defaulters': [[' Isidore Albert', ' 5 fraud_points']], 'loan_defaulter': []})
('CT96185', {'card_defaulters': [[' Danielle Bradshaw', ' 3 fraud_points']], 'loan_defaulter': []})
('CT74827', {'card_defaulters': [[' Hanna Boyer', ' 3 fraud_points']], 'loan_defaulter': []})
('CT98239', {'card_defaulters': [[' Sayuri Case', ' 4 fraud_points']], 'loan_defaulter': []})
('CT57141', {'card_defaulters': [[' Kaori Brown